## Устанавливаем Kaggle Dataset

In [74]:
import kagglehub
import tensorflow as tf
import os
import pandas as pd
from tensorflow import keras
from PIL import Image
import numpy as np


from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Download latest version
path = kagglehub.dataset_download("dmitryyemelyanov/chinese-traffic-signs")
images_path = os.path.join(path,'images')
annotations_csv_path = os.path.join(path,'annotations.csv')

print("Path to dataset files:", path)
print("Path to images:", images_path)

sign_categories = [3,4,5,7,11,16,17,26,30,35,43,55]
df = pd.read_csv(annotations_csv_path)

# Определяем форму изображения для наших картинок, так как иначе модель не сможет тренироваться
image_size = (32, 32)

# Проверяем, какие строки имеют категории из sign_categories
sign_images = []
categories = []

for index, row in df.iterrows():
    file_name = row['file_name']
    category = row['category']
    x1, x2 = row['x1'], row['x2']
    y1, y2 = row['y1'], row['y2']
    if category in sign_categories:
        sign_file = os.path.join(images_path, file_name)
        image = Image.open(sign_file)

        # Обрезаем изображение
        cropped_image_array = np.array(image)[y1:y2, x1:x2]

        # Изменяем размер обрезанного изображения

        resized_image = Image.fromarray(cropped_image_array).resize(image_size)
        
        sign_images.append(resized_image)
        categories.append(sign_categories.index(category))
categories = to_categorical(categories)

print("Количество знаков с совпадающими категориями:")
print(len(sign_images), len(categories))

# Конвертируем в необходимый для тренировки вид данных
sign_images = np.array(sign_images)
sign_images = sign_images / 255


Path to dataset files: C:\Users\k.bazhenov\.cache\kagglehub\datasets\dmitryyemelyanov\chinese-traffic-signs\versions\2
Path to images: C:\Users\k.bazhenov\.cache\kagglehub\datasets\dmitryyemelyanov\chinese-traffic-signs\versions\2\images
Количество знаков с совпадающими категориями:
2710 2710


In [83]:
X_train, X_test, y_train, y_test = train_test_split(sign_images, categories, test_size=0.2, random_state=42)

data_augmentation = tf.keras.Sequential([
  layers.RandomContrast([1.0, 10.0]),
  layers.RandomTranslation(height_factor=0.02, width_factor=0.02, fill_mode="constant")
])

model = tf.keras.models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', # (3,3) - фильтр
                        input_shape=(image_size[0],image_size[1],1)),
    layers.MaxPooling2D((2,2)), # фильтр (2,2) для пулинга
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, 'relu'),
    layers.Dense(len(sign_categories), 'softmax')
])
   

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# model.summary()
training_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=20
)

Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - acc: 0.4160 - loss: 1.7767 - val_acc: 0.8727 - val_loss: 0.9373
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.8905 - loss: 0.4838 - val_acc: 0.9336 - val_loss: 0.3530
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9583 - loss: 0.1802 - val_acc: 0.9705 - val_loss: 0.2137
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9594 - loss: 0.1572 - val_acc: 0.9446 - val_loss: 0.2181
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9692 - loss: 0.1256 - val_acc: 0.9779 - val_loss: 0.1162
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9842 - loss: 0.0672 - val_acc: 0.9889 - val_loss: 0.0711
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9820 - loss: 0.0799 - val_acc: 0.9926 - val_loss: 0.1238
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9841 - loss: 0.0740 - val_acc: 0.9779 - val_loss: 0.0954
Epoch 9/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9864 - loss: 